In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
customer= pd.read_csv(r'C:\Users\Dell\Downloads\customers_dataset.csv')
order_items= pd.read_csv(r'C:\Users\Dell\Downloads\order_items_dataset.csv')
order_payments= pd.read_csv(r'C:\Users\Dell\Downloads\order_payments_dataset.csv')
order_reviewer= pd.read_csv(r'C:\Users\Dell\Downloads\order_reviews_dataset.csv')
order= pd.read_csv(r'C:\Users\Dell\Downloads\orders_dataset.csv')
product_category= (r'C:\Users\Dell\Downloads\product_category_name_translation.csv')
products= pd.read_csv(r'C:\Users\Dell\Downloads\products_dataset.csv')

KẾT HỢP CÁC BẢNG LẠI VỚI NHAU

In [2]:
df= order.merge(order_items, on=['order_id'], how='left')
df= df.merge(order_payments, on=['order_id'], how= 'left')
df= df.merge(products, on=['product_id'], how= 'left')
df= df.merge(order_reviewer, on=['order_id'], how='left')
df= df.merge(customer, on=['customer_id'], how='left')

In [ ]:
df.info()

In [3]:
df['month']= pd.to_datetime(df['order_delivered_customer_date']).dt.month
df['year']= pd.to_datetime(df['order_delivered_customer_date']).dt.year

In [46]:
#quy đổi dữ liệu
df["month"]=df["month"].replace((1,2,3,4,5,6,7,8,9,10,11,12),('January','February' ,'March' ,'April' ,'May' ,'June' ,'July' ,'August' ,'September','October' ,'November' ,'December' ))

TÌNH HÌNH KINH DOANH CỦA CÔNG TY THEO THÁNG VÀ NĂM

In [ ]:
turnover= pd.pivot_table(df,index='month', columns='year', values='payment_value', aggfunc=np.sum, fill_value=0).reset_index(drop= False)
turnover

In [5]:
#doanh thu theo năm
turnover_year= df.groupby('year')['payment_value'].sum().reset_index()
xs,ys= turnover_year['year'], turnover_year['payment_value']

In [ ]:
plt.figure(figsize=(15,10))
plt.bar(xs,ys, color= sns.color_palette('Paired'));
plt.xlabel('year')
plt.ylabel('payment_value')
for x,y in zip(xs,ys):
    
        label = "{:.2f}".format(y)

        plt.annotate(label, # this is the text
                    (x,y), # these are the coordinates to position the label
                    textcoords="offset points", # how to position the text
                    xytext=(0,10), # distance from text to points (x,y)
                    ha='center') # horizontal alignment can be left, right or center

plt.show()

    

In [7]:
#doanh thu theo từng tháng
turnover_month= df.groupby('month')['payment_value'].sum().reset_index()
x,y= turnover_month['month'], turnover_month['payment_value']


In [ ]:
plt.figure(figsize=(20,12))
plt.plot(x,y)
plt.xlabel('month')
plt.ylabel('payment_value')
for a,b in zip(x,y):
    
        label = "{:.2f}".format(b)

        plt.annotate(label, # this is the text
                    (a,b), # these are the coordinates to position the label
                    textcoords="offset points", # how to position the text
                    xytext=(0,10), # distance from text to points (x,y)
                    ha='center') # horizontal alignment can be left, right or center

plt.show()

Top 10 sản phẩm bán chạy nhất đem lại doanh thu cao nhất cho công ty

In [ ]:
df.groupby('product_category_name')['payment_value'].sum().nlargest(10).reset_index()

SỐ ĐƠN HÀNG GIAO THÀNH CÔNG VÀ BỊ HỦY 

In [ ]:
df['order_status'].unique()

In [12]:
delivered_df= df.loc[df['order_status']=='delivered']
canceled_df= df.loc[df['order_status']=='canceled']

In [ ]:
plt.figure(figsize=(12,6))
sns.countplot(data=canceled_df,x='year');

In [ ]:
plt.figure(figsize=(12,6))
sns.countplot(data= delivered_df,x='year');

Hình thức thanh toán 

In [ ]:
plt.figure(figsize=(12,6))
sns.countplot(data=df, x='payment_type');

Số sao mà khách hàng đánh giá

In [46]:
star= df.groupby('review_score')['customer_id'].count().reset_index(name='count')
ax= star['review_score']
bx= star['count']

In [ ]:
plt.figure(figsize=(15,6))
plt.pie(bx, labels=ax ,autopct='%1.1f%%');

In [ ]:
#danh sách những sản phẩm đánh giá 1 và 2 sao
star12= df.loc[(df['review_score']== 1)|(df['review_score']==2)]
pd.crosstab(star12.product_category_name, star12.review_score)

Khách hàng chủ yếu tập trung ở những thành phố nào

In [ ]:
df['customer_city'].value_counts().nlargest(10).plot(kind='bar', figsize=(18,6), color= sns.color_palette('Set2'))
plt.xticks(rotation =0);